In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
dataPath = "/content/drive/MyDrive/rag/"

# New Section

라이브러리 설치 및 선언

In [3]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# 기존 라이브러리가 2월19일 기준 업데이트 되면서 pyarrow 라이브러리와 호환성 문제 발생 띠리사 호환라이브러리 정의
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.16.1 jsonlines==4.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00


In [4]:
import huggingface_hub
huggingface_hub.login()

In [5]:
import pandas as pd
import json

환경설정 (본인 구글드라이브 내 dataset 폴더

In [6]:
import jsonlines

In [7]:
from datasets import Dataset

1. 학습 데이터 불러오기 및 변환

!!! ★ 변경 포인트

In [8]:
# 데이터 경로 설정 및 불러오기
datasetName = "processing.csv"
jsonFileName = "hidak_processing.jsonl"

In [9]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기
    df = pd.read_csv(csv_file_path)

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['Question'], 'response': row['Answers']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

2. 파인튜닝용 포맷으로 변환

In [10]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
["<s>[INST] 안녕하세요. 가족중에 형이 안면비대칭 증상이 있어서 수술을 하려고 하네요. 엄청 심한 정도는 아니고 사진 찍으면, 티가 나는 정도인데요. 한쪽의 광대가 좀 더 튀어 나오고 턱은 그 반대 방향으로 살짝 치우쳐져있어서 안면비대칭이 티가 나기 때문에, 한쪽 광대를 조금 깎는 수술을 받는다고 하더군요. 얘기를 들어보니 뼈를깎고 밀어 넣는다고 하는데 제가 인터넷에서 예전에 어쩌다 본 글이 있습니다. 의사선생님께서 답변 해주었던 글로 기억 하는데 그런 수술을받고나면 뼈의 교합상태가 70%밖에 안된다고 하네요. 그런데 형이 취미로 권투를배우고 있고 좋아하기 때문에, 이 수술을 받고나서도 권투를 그대로 할수 있는지 걱정이 되고 정말로 뼈의 교합 강도가 70%밖에 안되는건지 궁금하네요. [/INST] 형의 안면비대칭 때문에 고민중이시군요.우선 뼈를 건드리는 수술은 바로 진행하는게 아니라, 사전에 CT촬영과 같은 검진을 통해 충분한 뼈의 검진을 하고 하기 때문에 안전합니다.그러나 성장환경에서 만들어진 뼈를 깎고 건드리는 수술이기 때문에 수술직후 초기에는 뼈의 움직임이 있고 수술을 했기 때문에 조심해야 하는것은 사실입니다만 시간이 지나고 회복을 하면서 다시 원래의 교합상태와 강성을 가지게 됩니다.실제로 저희 남자 직원중에서도 종합격투기를 취미로 하는 직원도 성형을 했는데 6개월이 지난 시점에서는 큰 불편함 없이 운동을 하고 있습니다.게다가 운동을 직업으로 삼는 선수들도 간혹 안와골절이라던가 코뼈의 골절이 생기는데 수술후 충분한 휴식과 재활을 거치면 다시 복귀해서 선수생활을 할 수 있듯이 안면비대칭 교정술을 받아도 충분한 회복후에는 취미로 복싱을하셔도 크게 문제는 없을것으로 보여집니다^^감사합니다.' </s>", "<s>[INST] 작년에 허벅지 지방흡입하고이번에 복부 지방흡입을 한달전에 했습니다허벅지 지방흡입할때는 이정도는 아니였는데복부 지방흡입하고서는 너무 통증이 심해서 잠을 잘 못자고 있습니다이 통증이 어느종도 지속이 되나요? [/INST]

Saving the dataset (0/1 shards):   0%|          | 0/135544 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 135544
})


3. huggingface 에 데이터 업로드 (★ 변경 포인트!)

In [11]:
indataset.push_to_hub("ssuengpp/hidak_all")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/136 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ssuengpp/hidak_all/commit/04b6ce33fb9a5f33598c1c3f2ed2b18a7dd8f7f4', commit_message='Upload dataset', commit_description='', oid='04b6ce33fb9a5f33598c1c3f2ed2b18a7dd8f7f4', pr_url=None, pr_revision=None, pr_num=None)

/bin/bash: line 1: nvidia-smi: command not found
